## Clustering usando SparkML


<p style='color: red'>El propósito de este cuaderno es mostrarte cómo usar SparkML para agrupar datos.


### __Tabla de contenidos__
<ol>
  <li>
    <a href="#Objectives">Objetivos
    </a>
  </li>
  <li>
    <a href="#Datasets">Conjuntos de datos
    </a>
  </li>
  <li>
    <a href="#Setup">Configuración
    </a>
    <ol>
      <li>
        <a href="#Installing-Required-Libraries">Instalación de librerías requeridas
        </a>
      </li>
      <li>
        <a href="#Importing-Required-Libraries">Importación de librería requeridas
        </a>
      </li>
    </ol>
  </li>
  <li>
    <a href="#Examples">Ejemplos
    </a>
    <ol>
      <li>
        <a href="#Task-1---Create-a-spark-session">Tarea 1 - Crear una sesión de Spark
        </a>
      </li>
      <li>
        <a href="#Task-2---Load-the-data-in-a-csv-file-into-a-dataframe">Tarea 2 - Cargar los datos de un archivo csv en un dataframe
        </a>
      </li>
      <li>
        <a href="#Task-3---Create-a-feature-vector">Tarea 3 - Crear un vector de características
        </a>
      </li>
      <li>
        <a href="#Task-4---Create-a-clustering-model">Tarea 4 - Crear un modelo de clustering
        </a>
      </li>
      <li>
        <a href="#Task-5---Print-Cluster-Centers">Tarea 5 - Imprimir centros de los clusters
        </a>
      </li>
    </ol>
  </li>
  <li>
    <a href="#Exercises">Ejercicios
    </a>
  </li>
  <ol>
    <li>
      <a href="#Exercise-1---Create-a-spark-session">Ejercicio 1 - Crear una sesión de Spark
      </a>
    </li>
    <li>
      <a href="#Exercise-2---Load-the-data-in-a-csv-file-into-a-dataframe">Ejercicio 2 - Cargar los datos de un archivo csv en un dataframe
      </a>
    </li>
    <li>
      <a href="#Exercise-3---Create-a-feature-vector">Ejercicio 3 - Crear un vector de características
      </a>
    </li>
    <li>
      <a href="#Exercise-4---Create-a-clustering-model">Ejercicio 4 - Crear un modelo de clustering
      </a>
    </li>
    <li>
      <a href="#Exercise-5---Print-Cluster-Centers">Ejercicio 5 - Imprimir centros de los clusters
      </a>
    </li>
  </ol>
</ol>


#### Objetivos

Después de completar este cuaderno serás capaz de:

 - Utilizar PySpark para conectarte a un clúster de Spark.
 - Crear una sesión de Spark.
 - Leer un archivo csv en un dataframe.
 - Usar el algoritmo KMeans para agrupar los datos.
 - Detener la sesión de Spark.
 
 
 


#### Conjuntos de datos

En este cuaderno utilizarás los siguientes conjuntos de datos:

 - Versión modificada del conjunto de datos de clientes mayoristas. Conjunto de datos original disponible en https://archive.ics.uci.edu/ml/datasets/Wholesale+customers 
 - Conjunto de datos de semillas. Disponible en https://archive.ics.uci.edu/ml/datasets/seeds


#### Configuración


Para este cuaderno, utilizaremos las siguientes librerías:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) para conectarse al clúster de Spark.


#### Instalación de librerías requeridas

Necesitas librerías como pyspark y findspark para trabajar este cuaderno.


Las siguientes librerías requeridas __no__ están preinstaladas. Necesitarás ejecutar la siguiente celda__ para instalarlas:


In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

#### Importando librerías requeridas

_Recomendamos importar todas las bibliotecas requeridas en un solo lugar (aquí):_


In [ ]:
# También puedes usar esta sección para suprimir advertencias generadas por tu código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifica el proceso de uso de Apache Spark con Python

import findspark
findspark.init()

# Importar funciones y clases para SparkML

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

# Crear una sesión de Spark
from pyspark.sql import SparkSession



#### Ejemplos


#### Tarea 1 - Crear una sesión de Spark


In [ ]:
# Crea una sesión de Spark
# Ignora cualquier advertencia generada por el comando SparkSession

spark = SparkSession.builder.appName("Clustering usando SparkML").getOrCreate()


#### Tarea 2 - Cargar los datos de un archivo csv en un dataframe


Descarga el archivo de datos


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/customers.csv


Carga el conjunto de datos en el dataframe de Spark


In [ ]:
# Usando la función spark.read.csv, cargamos los datos en un DataFrame.
# El parámetro header=True indica que la primera fila del archivo CSV contiene los nombres de las columnas.
# El parámetro inferSchema=True le indica a Spark que detecte automáticamente los tipos de datos de las columnas.

# Cargar el conjunto de datos de clientes
customer_data = spark.read.csv("customers.csv", header=True, inferSchema=True)


Imprime el esquema del conjunto de datos


In [ ]:
# Cada fila en este conjunto de datos representa un cliente. Las columnas indican los pedidos realizados por un cliente de:
# Alimentos frescos (Fresh_food), Leche (Milk), Abarrotes (Grocery) y Comida congelada (Frozen_Food).


In [ ]:
customer_data.printSchema()

Muestra las primeras 5 filas del conjunto de datos


In [ ]:
customer_data.show(n=5, truncate=False)

#### Tarea 3 - Crear un vector de características


In [ ]:
# Ensambla las características en una única columna de vector
feature_cols = ['Fresh_Food', 'Milk', 'Grocery', 'Frozen_Food']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Transforma los datos del cliente para incluir la nueva columna de características
customer_transformed_data = assembler.transform(customer_data)



### El algoritmo K-means

El algoritmo k-means pertenece a la familia de métodos de aprendizaje automático no supervisado, específicamente al área de clustering. Su objetivo es particionar un conjunto de observaciones en $k$ grupos (clusters), de tal forma que cada observación quede asignada al cluster con el centroide más cercano. 
El término "centroide" se refiere a un punto que representa el centro geométrico de todos los datos asignados a ese cluster. Usualmente, este centro se calcula como el promedio de las coordenadas de los puntos pertenecientes al grupo.

<img src="https://images.datacamp.com/image/upload/v1678462092/image7_a1777d39aa.png" width="700" height="600" alt="Imagen">


Debes indicarle al algoritmo KMeans cuántos clusters crear a partir de tus datos.




In [ ]:
number_of_clusters = 3

#### Tarea 4 - Crear un modelo de clustering


Crear un modelo de clustering KMeans


In [ ]:
kmeans = KMeans(k = number_of_clusters)


Entrena/ajusta el modelo en el conjunto de datos<br>


In [ ]:
modelo = kmeans.fit(customer_transformed_data)


#### Tarea 5 - Imprimir detalles de los clusters


Tu modelo ya está entrenado. Es hora de evaluar el modelo.


In [ ]:
# Realiza predicciones en el conjunto de datos
predictions = modelo.transform(customer_transformed_data)


In [ ]:
# Muestra el resultado
predictions.show(5)


Muestra cuántos clientes hay en cada cluster.


In [ ]:
predictions.groupBy('prediction').count().show()


In [ ]:
#Se detiene la sesión de spark
spark.stop()


#### Ejercicios


#### Ejercicio 1 - Crear una sesión de Spark


Crea una sesión de Spark con el nombre de la aplicación "Seed Clustering"


In [ ]:
spark = #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza SparkSession.builder

</details>


<details>
    <summary>Haz clic aquí para ver la solución</summary>
    
```python
spark = SparkSession.builder.appName("Seed Clustering").getOrCreate()
```
    
</details>


#### Ejercicio 2 - Cargar los datos de un archivo csv en un dataframe


In [ ]:
#download seed dataset
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/seeds.csv


Carga el conjunto de datos de semillas


In [ ]:

seed_data =  #TODO


<details>
    <summary>Haz clic aquí para una pista</summary>
    
Utiliza spark.read.csv

</details>


<details>
    <summary>Haz clic aquí para ver la solución</summary>
    
```python
seed_data = spark.read.csv("seeds.csv", header=True, inferSchema=True)
```
    
</details>


Imprime el esquema del conjunto de datos


In [ ]:
seed_data.printSchema()

Muestra las primeras 5 filas del conjunto de datos


In [ ]:
seed_data.show(n=5, truncate=False, vertical=True)

#### Ejercicio 3 - Crear un vector de características


Ensambla todas las columnas en un solo vector


In [ ]:
feature_cols =  #TODO
assembler =  #TODO
seed_transformed_data =  #TODO


<details>
    <summary>Haz clic aquí para una pista</summary>
    
Consulta la tarea 3
</details>


<details>
    <summary>Haz clic aquí para ver la solución</summary>
    
```python
feature_cols = ['area',
 'perimeter',
 'compactness',
 'length of kernel',
 'width of kernel',
 'asymmetry coefficient',
 'length of kernel groove']

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
seed_transformed_data = assembler.transform(seed_data)

```
    
</details>


#### Ejercicio 4 - Crear un modelo de clustering


Crea 7 clusters


In [ ]:
number_of_clusters =  #TODO
kmeans =  #TODO
model =  #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
utiliza el método kmeans.fit()
</details>


<details>
    <summary>Haz clic aquí para ver la solución</summary>
    
```python
number_of_clusters = 3
kmeans = KMeans(k = number_of_clusters)
model = kmeans.fit(seed_transformed_data)

```
    
</details>


#### Ejercicio 5 - Imprimir detalles de los clusters


In [ ]:
predictions =  #TODO

<details>
    <summary>Haz clic aquí para una pista</summary>
    
utiliza el método transform()
</details>


<details>
    <summary>Haz clic aquí para ver la solución</summary>
    
```python
predictions = model.transform(seed_transformed_data)
```
    
</details>


In [ ]:
predictions.show(n=5, truncate=False, vertical=True)

In [ ]:
predictions.groupBy('prediction').count().show()

In [ ]:
#stop spark session
spark.stop()

Te animamos a crear diferentes números de clusters utilizando el mismo conjunto de datos.
